In [1]:
import sqlite3
import numpy as np
import csv


grade_conversion = {
    '0': '0', 
    '6B': 'V4', 
    '6B+': 'V4', 
    '6C': 'V5', 
    '6C+': 'V5', 
    '7A': 'V6', 
    '7A+': 'V7', 
    '7B': 'V8', 
    '7B+': 'V8', 
    '7C': 'V9', 
    '7C+': 'V10', 
    '8A' : 'V11', 
    '8A+': 'V12', 
    '8B' : 'V13', 
    '8B+': 'V14'
}



# A mapping of the holds on the Moonboard 2016 setup to their type (A, B or school)
# Is only used to have a list of all the holds. I am not interested in the types in this project.

holdmapping = {'2016': {'A10': ['school'],
          'A11': ['school'],
          'A12': ['A'],
          'A13': ['school'],
          'A14': ['B'],
          'A15': ['A'],
          'A16': ['school'],
          'A18': ['B'],
          'A5': ['A'],
          'A9': ['B'],
          'B10': ['B'],
          'B11': ['A'],
          'B12': ['B'],
          'B13': ['A'],
          'B15': ['B'],
          'B16': ['A'],
          'B18': ['A'],
          'B3': ['A'],
          'B4': ['school'],
          'B6': ['B'],
          'B7': ['school'],
          'B8': ['A'],
          'B9': ['A'],
          'C10': ['A'],
          'C11': ['A'],
          'C12': ['school'],
          'C13': ['B'],
          'C14': ['A'],
          'C15': ['school'],
          'C16': ['B'],
          'C18': ['school'],
          'C5': ['B'],
          'C6': ['A'],
          'C7': ['school'],
          'C8': ['B'],
          'C9': ['school'],
          'D10': ['school'],
          'D11': ['B'],
          'D12': ['A'],
          'D13': ['school'],
          'D14': ['A'],
          'D15': ['B'],
          'D16': ['school'],
          'D17': ['A'],
          'D18': ['B'],
          'D3': ['school'],
          'D5': ['A'],
          'D6': ['school'],
          'D7': ['B'],
          'D8': ['school'],
          'D9': ['A'],
          'E10': ['A'],
          'E11': ['A'],
          'E12': ['B'],
          'E13': ['A'],
          'E14': ['B'],
          'E15': ['A'],
          'E16': ['B'],
          'E18': ['A'],
          'E6': ['B'],
          'E7': ['A'],
          'E8': ['B'],
          'E9': ['B'],
          'F10': ['A'],
          'F11': ['B'],
          'F12': ['A'],
          'F13': ['B'],
          'F14': ['A'],
          'F15': ['school'],
          'F16': ['A'],
          'F5': ['A'],
          'F6': ['school'],
          'F7': ['A'],
          'F8': ['school'],
          'F9': ['school'],
          'G10': ['B'],
          'G11': ['school'],
          'G12': ['A'],
          'G13': ['A'],
          'G14': ['B'],
          'G15': ['B'],
          'G16': ['school'],
          'G17': ['B'],
          'G18': ['A'],
          'G2': ['A'],
          'G4': ['B'],
          'G6': ['B'],
          'G7': ['school'],
          'G8': ['A'],
          'G9': ['B'],
          'H10': ['B'],
          'H11': ['A'],
          'H12': ['B'],
          'H13': ['B'],
          'H14': ['school'],
          'H15': ['school'],
          'H16': ['B'],
          'H18': ['school'],
          'H5': ['B'],
          'H7': ['school'],
          'H8': ['B'],
          'H9': ['school'],
          'I10': ['B'],
          'I11': ['A'],
          'I12': ['school'],
          'I13': ['A'],
          'I14': ['B'],
          'I15': ['B'],
          'I16': ['A'],
          'I18': ['B'],
          'I4': ['A'],
          'I5': ['school'],
          'I6': ['A'],
          'I7': ['B'],
          'I8': ['school'],
          'I9': ['B'],
          'J10': ['A'],
          'J11': ['school'],
          'J12': ['B'],
          'J13': ['B'],
          'J14': ['school'],
          'J16': ['B'],
          'J2': ['A'],
          'J5': ['B'],
          'J6': ['A'],
          'J7': ['B'],
          'J8': ['A'],
          'J9': ['A'],
          'K10': ['school'],
          'K11': ['A'],
          'K12': ['school'],
          'K13': ['school'],
          'K14': ['B'],
          'K16': ['B'],
          'K18': ['A'],
          'K5': ['A'],
          'K6': ['school'],
          'K7': ['school'],
          'K8': ['school'],
          'K9': ['B']}}


In [2]:
# This snippet extracts all problem moves and grades from database

db = 'moon_problems.db'
conn = sqlite3.connect(db)
c = conn.cursor()
c.execute("SELECT pm.Problem, pm.Position, p.Grade FROM problemMoves_2016 pm INNER JOIN problems p ON pm.Problem = p.Id")
problem_grade = {}
problem_moves = {}
problem_lengths = []   # for histogramming
problem_grades = []    # for histogramming
id_current = 0
id_previous = 0
holds = []
for entry in c.fetchall():
    id_previous = id_current
    id_current = entry[0]
    if id_previous != id_current:
        problem_moves[id_previous] = holds
        holds = []
        problem_grade[id_current] = entry[2]
    holds.append(entry[1])
problem_moves[id_current] = holds

del problem_moves[0]   #  the previous loop added an empty list as moves for problem ID 0, so I am removing it 




In [3]:
# This snippet determines the difficulty of each hold by counting the number of times they appear in easy / medium / holds
def upsample_holds(holds):
    # This function duplicates the holds in a climb until a size of 12 holds is reached
    # This is necessary because otherwise the easy climbs having more holds total, will overwhelm the classification
    holds_original = holds
    length = len(holds)
    if length < 12:
        for i in range(12-length):
            n = np.random.randint(len(holds_original))
            holds.append(holds_original[n])
    return holds


av_holds_easy = 0
av_holds_medium = 0
av_holds_hard = 0
for problem in problem_moves:
    if grade_conversion[problem_grade[problem]] in ['V4', 'V5']:
        av_holds_easy += len(problem_moves[problem])
    elif grade_conversion[problem_grade[problem]] in ['V6', 'V7']:
        av_holds_medium += len(problem_moves[problem])
    else:
        av_holds_hard += len(problem_moves[problem])

av_holds_easy = av_holds_easy/len(problem_moves)
av_holds_medium = av_holds_medium/len(problem_moves)
av_holds_hard = av_holds_hard/len(problem_moves)

holdsetup = []

for hold in holdmapping['2016']:
    holdsetup.append(hold)

hold_difficulty = {}
for hold in holdsetup:
    easy = 0
    medium = 0
    hard = 0
    used = 0
    for problem in problem_moves:
        holds_problem = upsample_holds(problem_moves[problem])
        for hold_local in holds_problem:
            if hold == hold_local:
                used += 1
                if grade_conversion[problem_grade[problem]] in ['V4', 'V5']:
                    easy += 1
                elif grade_conversion[problem_grade[problem]] in ['V6', 'V7']:
                    medium += 1
                elif grade_conversion[problem_grade[problem]] in ['V8', 'V9+']:
                    hard += 1
    rebalance_easy = easy/used
    rebalance_medium = medium/used
    rebalance_hard = hard/used
    if max(rebalance_easy, rebalance_medium, rebalance_hard) == rebalance_easy:
        hold_difficulty[hold] = 0.33
    elif  max(rebalance_easy, rebalance_medium, rebalance_hard) == rebalance_medium:
        hold_difficulty[hold] = 0.66
    else:
        hold_difficulty[hold] = 1
    

w = csv.writer(open("hold_difficulty.csv", "w", newline=''))
for key, val in hold_difficulty.items():
    w.writerow([key, val])